<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=170 style="padding: 10px"> 
<b>Little Demo: datashader</b> <br>
Contact author(s): Leanne Guy, Melissa Graham <br>
Last verified to run: 2023-05-08 <br>
LSST Science Pipelines version: Weekly 2023_07 <br>
Container Size: medium <br>

Traditional methods use static plots, or partially-transparent points, become confusion-limited with millions of objects.

The Rubin Science Platform will support third-party packages such as `datashader` to enable interactive plots to showcase millions of LSST detections.

For a more detailed tutorial on visualization techniques for catalog data, see <a href="https://github.com/rubin-dp0/tutorial-notebooks">tutorial notebook</a> 06b.

## 1. Set up

Import packages.

In [ ]:
from lsst.rsp import get_tap_service
import datashader as dsh
from bokeh.io import output_notebook
import holoviews as hv
from holoviews import streams
from holoviews.operation.datashader import dynspread, datashade

Set parameters to show plots in the notebook.

In [ ]:
hv.extension('bokeh')
output_notebook()

Instantiate the TAP service.

In [ ]:
service = get_tap_service()

For a more detailed tutorial on the TAP service, see <a href="https://github.com/rubin-dp0/tutorial-notebooks">tutorial notebook</a> 02.

Query for and return data for at least 1 million Objects (takes ~1 minute).

In [ ]:
query = "SELECT coord_ra, coord_dec, objectId, r_extendedness, "\
        "scisql_nanojanskyToAbMag(g_cModelFlux) AS mag_g_cModel, "\
        "scisql_nanojanskyToAbMag(r_cModelFlux) AS mag_r_cModel, "\
        "scisql_nanojanskyToAbMag(i_cModelFlux) AS mag_i_cModel "\
        "FROM dp02_dc2_catalogs.Object "\
        "WHERE CONTAINS(POINT('ICRS', coord_ra, coord_dec),"\
        "CIRCLE('ICRS', 55.65, -40, 1)) = 1 " \
        "AND detect_isPrimary = 1 "\
        "AND scisql_nanojanskyToAbMag(r_cModelFlux) < 27.0 "\
        "AND r_extendedness IS NOT NULL"
data = service.search(query).to_table().to_pandas()
print(len(data))

Compute _g-r_ and _r-i_ colors for all objects.

In [ ]:
data['gmr'] = data['mag_g_cModel'] - data['mag_r_cModel']
data['rmi'] = data['mag_r_cModel'] - data['mag_i_cModel']

## 2. Plot with datashader

Define the plot's points and bounding box.

In [ ]:
points = hv.Points((data['gmr'], data['rmi']))
boundsxy = (0, 0, 0, 0)
box = streams.BoundsXY(source=points, bounds=boundsxy)
bounds = hv.DynamicMap(lambda bounds: hv.Bounds(bounds), streams=[box])

Define the plot itself, and display it.

In [ ]:
p = dynspread(datashade(points, cmap="Viridis"))
p = p.opts(width=800, height=300, padding=0.05, show_grid=True,
           xlim=(-2.0, 7.0), ylim=(-5.0, 3.0), xlabel="(g-r)", ylabel="(r-i)")#,
           # tools=['box_select', 'lasso_select'])
p * bounds

Use the icon with the magnifying glass to zoom in, and notice how the plot automatically adjusts from heatmap to individual points, allowing the million points to be visualized all together.

Use the two-arrows-in-a-circle icon to reset the plot.